# kNN & Clustering

__Суммарное количество баллов: 10__

## kNN и рак (3 балла)

В этом части домашнего задания Вам предлагается при помощи классификации методом k ближайших соседей научиться отличать тип опухоли в организме

In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
import pandas
from typing import Tuple, List

### Данные


Реализуйте метод `read_cancer_dataset` . Он принимает на вход путь к набору данных и возвращает выборку `X` и соответствующие метки `y`.

In [ ]:
CANCER_DATA_PATH = './hw_knn_data/cancer.csv'

In [ ]:
def read_cancer_dataset(path_to_csv: str) -> Tuple[pd.DataFrame, np.array]:
    # Возвращает пару из X и y. X - массив векторов. y - соответствующие векторам метки
    data: pd.DataFrame = pd.read_csv(path_to_csv)
    return data.drop('label', axis = 1), pd.factorize(data['label'])[0]

In [ ]:
X_cancer, y_cancer = read_cancer_dataset(CANCER_DATA_PATH)

Начиная работать с данными, нам необходимо их предобработать и подготовить. В частности, нам необходимо разделить выборку на две: тренировочную и тестовую. Тренировочная выборка необходима для обучения алгоритма, а тестовая для проверки результатов обучения. Обычно используют коэффициент разделения `0.7`.

In [ ]:
def train_test_split(X: pd.DataFrame, y: np.array, ratio: float) -> Tuple[pd.DataFrame, np.array, pd.DataFrame, np.array]:
    # Возвращает X_train, y_train, X_test, y_test
    # X_train и X_test - массив векторов - две части массива X, разделенного в состветсви с коэффициентом ratio
    # y_train и y_test - соответствующие X_train и X_test метки классов
    treshold: int = int(X.shape[0] * ratio)
    return X[:treshold], y[:treshold], X[treshold:], y[treshold:]

### Метрики

Также прежде чем приступать к решению задачи, нам необходимо определиться с метриками, которые позволят нам оценить полученное решение. Для задач классификации мы можем использовать precision, recall и accuracy. Эти метрики считаются для каждого класса. 

__Precision__ отражает то, насколько редко мы ошибаемся, когда говорим, что объект пренадлежит к классу. 

__Recall__ же отражает то, насколько редко классификатор неправильно классифицирует объекты данного класса.

__Accuracy__ отражает то, какую часть выборки классификатор отнес к правильному классу.

In [ ]:
def get_precision_recall_accuracy(y_pred: np.array, y_true: np.array) -> Tuple[List[float], List[float], float]:
    # Возвращает precision, recall и accuracy
    # precision - набор значений метрики precision для каждого класса
    # recall - набор значений метрики recall для каждого класса
    # accuracy - число, отражающее общую точность предсказания
    precision, recall = [], []
    classes = np.unique(y_true)
    
    for cls in classes:
        TP = 0

        for pred, true in zip(y_pred, y_true):
            if pred == cls and pred == true:
                TP += 1
        
        precision.append(TP / len(list(filter(lambda y: y == cls, y_pred))))
        recall.append(TP / len(list(filter(lambda y: y == cls, y_true))))
        
    accuracy = len(list(filter(lambda y: y[0] == y[1], zip(y_pred, y_true)))) / len(y_pred)

    return precision, recall, accuracy

Теперь, имея этот метод, мы можем построить кривые зависимости Precision, Recall и Accuracy от параметра `k`

In [ ]:
def plot_precision_recall(X_train, y_train, X_test, y_test, max_k=30):
    ks = list(range(1, max_k + 1))
    classes = len(np.unique(list(y_train) + list(y_test)))
    precisions = [[] for _ in range(classes)]
    recalls = [[] for _ in range(classes)]
    accuracies = []
    for k in ks:
        classifier = KNearest(k)
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        precision, recall, acc = get_precision_recall_accuracy(y_pred, y_test)
        for c in range(classes):
            precisions[c].append(precision[c])
            recalls[c].append(recall[c])
        accuracies.append(acc)
    def plot(x, ys, ylabel, legend=True):        
        plt.figure(figsize = (12, 3))
        plt.xlabel("K")
        plt.ylabel(ylabel)
        plt.xlim(x[0], x[-1])
        plt.ylim(np.min(ys)-0.01, np.max(ys)+0.01)
        for cls, cls_y in enumerate(ys):
            plt.plot(x, cls_y, label="Class " + str(cls))
        if legend:
            plt.legend()
        plt.tight_layout()
        plt.show()
    
    plot(ks, recalls, "Recall")
    plot(ks, precisions, "Precision")
    plot(ks, [accuracies], "Accuracy", legend=False)

Также для оценки качества классификации построим __ROC-кривую__. Она отражает зависимость __True Positive Rate__ (TPR) от __False Positive Rate__ (FPR) для заранее фиксированного класса. Чем график выше побочной диагонали - тем лучше.

In [ ]:
def plot_roc_curve(X_train, y_train, X_test, y_test, max_k=30):
    positive_samples = sum(1 for y in y_test if y == 0)
    ks = list(range(1, max_k + 1))
    curves_tpr = []
    curves_fpr = []
    colors = []
    for k in ks:
        colors.append([k / ks[-1], 0, 1 - k / ks[-1]])
        knearest = KNearest(k)
        knearest.fit(X_train, y_train)
        p_pred = [p[0] for p in knearest.predict_proba(X_test)]
        tpr = []
        fpr = []
        for w in np.arange(-0.01, 1.02, 0.01):
            y_pred = [(0 if p > w else 1) for p in p_pred]
            tpr.append(sum(1 for yp, yt in zip(y_pred, y_test) if yp == 0 and yt == 0) / positive_samples)
            fpr.append(sum(1 for yp, yt in zip(y_pred, y_test) if yp == 0 and yt != 0) / (len(y_test) - positive_samples))
        curves_tpr.append(tpr)
        curves_fpr.append(fpr)
    plt.figure(figsize = (7, 7))
    for tpr, fpr, c in zip(curves_tpr, curves_fpr, colors):
        plt.plot(fpr, tpr, color=c)
    plt.plot([0, 1], [0, 1], linestyle="--")
    plt.xlabel("False positive rate")
    plt.ylabel("True positive rate")
    plt.xlim(-0.01, 1.01)
    plt.ylim(-0.01, 1.01)
    plt.tight_layout()
    plt.show()

## KNN
Осталось реализовать сам классификатор. Реализуйте его, используя KD-дерево. (При желании можно воспользоваться библиотечной реализацией дерева)

Метод `__init__` принимает на вход количество соседей, по которым предсказывается класс, и размер листьев KD-дерева.

Метод `fit` должен по набору данных и меток "обучать" классификатор. 

Метод `predict_proba` должен предсказывать вероятности классов для заданного набора данных основываясь на классах соседей

In [ ]:
from sklearn.neighbors import KDTree

class KNearest:
    def __init__(self, n_neighbors=5, leaf_size=30):
        self.__n_neighbors = n_neighbors
        self.__leaf_size = leaf_size
    
    def fit(self, X: pd.DataFrame, y: np.array) -> None:
        self.__test_data = X.copy()
        self.__test_data['label'] = y
        self.__classes = np.unique(y)

        self.__tree = KDTree(X, self.__leaf_size)
        
    def predict_proba(self, X: pd.DataFrame) -> np.array:
        # Возвращает матрицу, в которой строки соответствуют элементам X, а столбцы - классам. На пересечении строки и столбца должна быть указана вероятность того, что элемент относится к классу
        # Вероятность рассчитывается как количество ближайших соседей с данным классом деленное на общее количество соседей
        result = np.zeros((X.shape[0], len(self.__classes)))

        indexes = self.__tree.query(X, self.__n_neighbors, return_distance=False)

        for i, row in enumerate(indexes):
            for idx in row:
                for j in range(len(self.__classes)):
                    if self.__test_data.iloc[idx]['label'] == self.__classes[j]:
                        result[i][j] += 1
                
                for j in range(len(result[i])):
                    result[i][j] /= self.__n_neighbors

        return result
        
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

Наконец, протестируем наш классификатор на датасете _cancer_

In [ ]:
X_train, y_train, X_test, y_test = train_test_split(X_cancer, y_cancer, 0.9)
plot_precision_recall(X_train, y_train, X_test, y_test)
plot_roc_curve(X_train, y_train, X_test, y_test, max_k=10)



Проанализируйте полученные графики. Какой параметр `k` кажется лучшим? Какая из метрик лучше всего отражает качество модели? 

_Ваш ответ_ Лучшим вариантом кажется, k = 10, т.к. при равных precision и recall для первого класса на этом значении достигаются высокие показатели этих метрик для нулевого класса, а так же самая высокая accuracy. Кажется, что в данном случае, accuracy лучше всего показывает качество модели, так как её значения примерно коррелируют с точностью предсказаний нулевого класса

## Clustering

В этой части домашнего задания предлагается реализовать три различных метода кластеризации, понять, в каких случаях стоит применять те или иные методы.

In [ ]:
from sklearn.neighbors import KDTree
from sklearn.datasets import make_blobs, make_moons, make_swiss_roll
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
import cv2
from collections import deque

In [ ]:
def visualize_clasters(X, labels):
    unique_labels = np.unique(labels)
    unique_colors = np.random.random((len(unique_labels), 3))
    colors = [unique_colors[l] for l in labels]
    plt.figure(figsize=(9, 9))
    plt.scatter(X[:, 0], X[:, 1], c=colors)
    plt.show()

def clusters_statistics(flatten_image, cluster_colors, cluster_labels):
    fig, axes = plt.subplots(3, 2, figsize=(12, 16))
    for remove_color in range(3):
        axes_pair = axes[remove_color]
        first_color = 0 if remove_color != 0 else 2
        second_color = 1 if remove_color != 1 else 2
        axes_pair[0].scatter([p[first_color] for p in flatten_image], [p[second_color] for p in flatten_image], c=flatten_image, marker='.')
        axes_pair[1].scatter([p[first_color] for p in flatten_image], [p[second_color] for p in flatten_image], c=[cluster_colors[c] for c in cluster_labels], marker='.')
        for a in axes_pair:
            a.set_xlim(0, 1)
            a.set_ylim(0, 1)
    plt.show()

Генерируем два синтетических набора данных для кластеризации. Далее будем тестировать наши алгоритмы на них.

In [ ]:
X_1, true_labels = make_blobs(400, 2, centers=[[0, 0], [-4, 0], [3.5, 3.5], [3.5, -2.0]])
visualize_clasters(X_1, true_labels)
X_2, true_labels = make_moons(400, noise=0.075)
visualize_clasters(X_2, true_labels)

### K-means (3 балла)

Первый метод, который предлагается реализовать - метод K средних.

__Описание методов__

`fit(X, y=None)` ищет и запоминает в `self.centroids` центроиды кластеров для набора данных.
`predict(X)` для каждого элемента из `X` возвращает номер кластера, к которому относится данный элемент.

__Инициализация кластеров__

Есть несколько вариантов инициализации кластеров. Нужно реализовать их все:
1. `random` - центроиды кластеров являются случайными точками
2. `sample` - центроиды кластеров выбираются случайно из набора данных
3. `k-means++` - центроиды кластеров инициализируются при помощи метода K-means++


In [ ]:
from sklearn.preprocessing import minmax_scale
from functools import reduce

class KMeans:
    def __init__(self, n_clusters, init="random", max_iter=300):
        if init not in ['random', 'sample', 'k-means++']:
            raise ValueError('"init" must be either "random", "sample" or "k-means++"')
        
        self.__n_clusters = n_clusters
        self.__init = init
        self.__max_iter = max_iter
        
    def fit(self, X, y=None):
        dimension = len(X[0])
        self.__data = minmax_scale(X)
        
        if self.__init == 'random':
            self.__centroids = np.random.random((self.__n_clusters, dimension))
        elif self.__init == 'sample':
            indexes = np.random.choice(range(len(self.__data)), self.__n_clusters)
            self.__centroids = np.array(list(map(lambda idx: self.__data[idx], indexes)))
        elif self.__init == 'k-means++':
            self.__centroids = np.zeros((self.__n_clusters, dimension))
            self.__centroids[0] = self.__data[np.random.randint(0, len(self.__data))]
            
            for new_centroid_idx in range(1, self.__n_clusters):
                distances = np.array(list(map( # Массив с минимальным расстоянием для каждого примера
                    lambda example: min(list(map( # Проходимся по всем уже найденым центроидам и находим ближайший
                        lambda centroid_idx: reduce( # Проходимся по всем измерениям и считаем квадрат евклидова расстояния
                            lambda acc, d: acc + (example[d] - self.__centroids[centroid_idx][d])**2,
                            range(dimension),
                            0
                        ),
                        range(new_centroid_idx)
                    ))),
                    self.__data
                )))

                probabilities = distances / np.sum(distances)
                # TODO: Сделать модель вероятностной
                self.__centroids[new_centroid_idx] = self.__data[np.argmax(probabilities)]

        raise NotImplementedError()
    
    def predict(self, X):
        raise NotImplementedError()

Протестируйте алгоритм на синтетических данных. При необходимости подберите гиперпараметры для достижения лучшего качества кластеризации.

In [ ]:
kmeans = KMeans(n_clusters=4, init='k-means++')
kmeans.fit(X_1)
labels = kmeans.predict(X_1)
visualize_clasters(X_1, labels)

kmeans = KMeans(n_clusters=2)
kmeans.fit(X_2)
labels = kmeans.predict(X_2)
visualize_clasters(X_2, labels)

### DBScan (4 балла)
В отличии от K-means, DBScan не позволяет задать количество кластеров, на которое будут разбиты данные. Руководствуясь геометрической интерпретацией, он позволяет выделять кластеры более сложной формы.

__Описание методов__

`fit_predict(X, y=None)` для каждого элемента `X` возвращает метку кластера, к которому он относится.

__Возможные метрики__

* `euclidean`
* `manhattan`
* `chebyshev`

__Для быстрого поиска соседей используйте `sklearn.neighbors.KDTree`__

In [ ]:
class DBScan:
    def __init__(self, eps=0.5, min_samples=5, leaf_size=40, metric="euclidean"):
        raise NotImplementedError()
        
    def fit_predict(self, X, y=None):
        raise NotImplementedError()

Протестируйте алгоритм на синтетических данных. При необходимости подберите гиперпараметры для достижения лучшего качества кластеризации.

In [ ]:
dbscan = DBScan()
labels = dbscan.fit_predict(X_1)
visualize_clasters(X_1, labels)

dbscan = DBScan()
labels = dbscan.fit_predict(X_2)
visualize_clasters(X_2, labels)

Проанализируйте полученные результаты. 

Какой метод лучше справился с кластеризацией? Почему? 

Сравните значения метрик  `Davies-Bouldin index` и `Silhouette score` для определения качества кластеризации. 

Какие значения метрики свидетельствуют о хорошей кластеризации - большие или маленькие?

_Ваш ответ_

In [ ]:
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import silhouette_score

In [ ]:
# YOUR_CODE